In [2]:
# Question 1(a): March 2026 CAISO Peak Load Point Forecast

import pandas as pd
import numpy as np

df = pd.read_csv("/content/CAISOHourlyLoadCSV.csv")
df.columns = [c.strip() for c in df.columns]

possible_load_cols = ["Load_MW", "Load MW", "Load", "MW", "load_mw", "CAISO_Load_MW"]
load_col = next((c for c in possible_load_cols if c in df.columns), None)

if load_col is None:
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [c for c in numeric_cols if c.lower() not in ["hour", "year", "month", "day"]]
    load_col = numeric_cols[0]

df["Date"] = pd.to_datetime(df["Date"])
df["Hour"] = df["Hour"].astype(int)
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["timestamp_end"] = df["Date"] + pd.to_timedelta(df["Hour"], unit="h")

march_2025 = df[(df["Year"] == 2025) & (df["Month"] == 3)]
peak_idx = march_2025[load_col].idxmax()
peak_row = march_2025.loc[peak_idx]

march_2025_peak = float(peak_row[load_col])
peak_date = peak_row["Date"].date()
peak_hour = int(peak_row["Hour"])

growth_rate = 0.015
march_2026_peak_forecast = march_2025_peak * (1 + growth_rate)

print(f"March 2025 peak load: {march_2025_peak:,.2f} MW")
print(f"Peak occurred on {peak_date}, Hour {peak_hour} (hour-ending)")
print(f"March 2026 peak load forecast (unrounded): {march_2026_peak_forecast:,.2f} MW")
print(f"March 2026 peak load forecast (rounded): {int(round(march_2026_peak_forecast)):,} MW")

March 2025 peak load: 28,127.03 MW
Peak occurred on 2025-03-14, Hour 11 (hour-ending)
March 2026 peak load forecast (unrounded): 28,548.94 MW
March 2026 peak load forecast (rounded): 28,549 MW


In [4]:
# Question 1(b): Forecasted Date of March 2026 CAISO Peak Load

import pandas as pd

df = pd.read_csv("/content/CAISOHourlyLoadCSV.csv")

df["Date"] = pd.to_datetime(df["Date"])
df["Hour"] = df["Hour"].astype(int)
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month

march_2025 = df[(df["Year"] == 2025) & (df["Month"] == 3)]
peak_idx = march_2025["CAISO Load (MW)"].idxmax()
peak_row = march_2025.loc[peak_idx]

peak_date_2025 = peak_row["Date"].date()
forecast_peak_date_2026 = peak_date_2025.replace(year=2026)

print(f"March 2025 peak date: {peak_date_2025}")
print(f"Forecasted March 2026 peak date: {forecast_peak_date_2026}")

March 2025 peak date: 2025-03-14
Forecasted March 2026 peak date: 2026-03-14


In [5]:
# Question 1(c): Backtest on Historical Data + Error Metric for Monthly Peak MW

import pandas as pd
import numpy as np

df = pd.read_csv("/content/CAISOHourlyLoadCSV.csv")
df["Date"] = pd.to_datetime(df["Date"])
df["Hour"] = df["Hour"].astype(int)

load_col = "CAISO Load (MW)"

df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["YearMonth"] = df["Date"].dt.to_period("M").astype(str)

monthly_peaks = (
    df.groupby("YearMonth", as_index=False)[load_col]
      .max()
      .rename(columns={load_col: "Peak_MW"})
)

monthly_peaks["t"] = np.arange(len(monthly_peaks))

min_train_months = 3
rows = []

for i in range(min_train_months, len(monthly_peaks)):
    train = monthly_peaks.iloc[:i].copy()
    test = monthly_peaks.iloc[i].copy()

    X = train["t"].values
    y = train["Peak_MW"].values

    b, a = np.polyfit(X, y, 1)  # y = a + b*t
    yhat = a + b * test["t"]

    rows.append({
        "YearMonth": test["YearMonth"],
        "Actual_Peak_MW": float(test["Peak_MW"]),
        "Forecast_Peak_MW": float(yhat),
        "Error_MW": float(yhat - test["Peak_MW"]),
        "AbsError_MW": float(abs(yhat - test["Peak_MW"])),
        "APE_pct": float(abs(yhat - test["Peak_MW"]) / test["Peak_MW"] * 100.0)
    })

bt = pd.DataFrame(rows)

mae = bt["AbsError_MW"].mean()
mape = bt["APE_pct"].mean()
rmse = float(np.sqrt(np.mean((bt["Error_MW"].values) ** 2)))

print("Monthly peak backtest (rolling-origin, linear trend on prior monthly peaks)")
print(f"Backtest months: {bt.shape[0]}")
print(f"MAE (MW): {mae:,.1f}")
print(f"RMSE (MW): {rmse:,.1f}")
print(f"MAPE (%): {mape:,.2f}")

display(bt)

Monthly peak backtest (rolling-origin, linear trend on prior monthly peaks)
Backtest months: 9
MAE (MW): 4,113.9
RMSE (MW): 5,578.2
MAPE (%): 11.64


,YearMonth,Actual_Peak_MW,Forecast_Peak_MW,Error_MW,AbsError_MW,APE_pct
0,2025-02,29126.53,30030.890000,904.360000,904.360000,3.104936
1,2025-03,28127.03,29496.450000,1369.420000,1369.420000,4.868697
2,2025-04,28444.29,28499.526000,55.236000,55.236000,0.194190
3,2025-05,36265.24,28287.430000,-7977.810000,7977.810000,21.998503
4,2025-06,36284.07,32663.015714,-3621.054286,3621.054286,9.979736
5,2025-07,39655.38,35145.145357,-4510.234643,4510.234643,11.373576
6,2025-08,43922.82,38123.051111,-5799.768889,5799.768889,13.204455
7,2025-09,42416.44,41716.998000,-699.442000,699.442000,1.648988
8,2025-10,31473.68,43561.730909,12088.050909,12088.050909,38.406856


## Question 1 part c:

To forecast the March 2026 peak load, I followed a deliberately simple and transparent analytical workflow focused on modeling monthly peak demand rather than full hourly load profiles. The objective was to produce a defensible point forecast while maintaining reproducibility and interpretability.

I began by aggregating the hourly CAISO load data into a time series of monthly peak loads, defined as the maximum observed hourly load within each calendar month. This transformation aligns directly with the forecasting target, which is the monthly peak rather than average or total load.

As an interim modeling step, I implemented a rolling-origin backtest using a linear trend model fitted to historical monthly peak loads. At each step of the backtest, the model was trained using all available prior monthly peak observations and then used to forecast the peak for the subsequent month. This approach mirrors an operational forecasting setting in which only past information is available at the time of forecast generation.

Model performance was evaluated using three standard error metrics. Mean Absolute Error was used to measure the average magnitude of forecast errors in megawatts. Root Mean Squared Error was used to place greater weight on larger forecast errors. Mean Absolute Percentage Error was used to provide a scale-free measure of relative error across months with differing load levels.

Across the nine-month backtest period, the model achieved a mean absolute error of 4,113.9 MW, a root mean squared error of 5,578.2 MW, and a mean absolute percentage error of 11.64 percent. Forecast errors were larger during summer months, reflecting the impact of extreme temperatures and nonlinear demand responses that are not captured by a simple trend-based model. Forecast accuracy was materially better during spring and shoulder-season months, including March.

Based on these results, I concluded that a parsimonious growth-based projection anchored to recent March peak observations is appropriate for forecasting the March 2026 peak load. The backtest provides quantitative evidence supporting the use of a modest year-over-year growth assumption and offers a benchmark for expected forecast uncertainty.

## Question 1 Part D:

The final forecasting approach is a growth-based projection of the historical March monthly peak load. The forecast targets the maximum hourly CAISO system load in March 2026 and is anchored to the observed March 2025 peak load derived directly from the provided hourly dataset.

Formally, the model forecasts the March 2026 peak load as the March 2025 peak multiplied by a fixed year-over-year growth factor. Let L_2025 denote the observed March 2025 peak load and g denote the assumed annual growth rate. The forecasted March 2026 peak load L_2026 is given by L_2026 = L_2025 × (1 + g).

The sole model parameter is the growth rate g, which was set to 1.5 percent. This value reflects modest structural demand growth over a one-year horizon and is consistent with the magnitude of historical variation observed in recent monthly peak loads. The appropriateness of a simple growth-based model for a short-horizon March forecast was supported by a rolling-origin backtest on historical monthly peak data, which showed materially lower forecast errors during spring and shoulder-season months compared to summer months dominated by extreme temperature effects.

This approach was selected as the final model due to its transparency, reproducibility, and alignment with the forecasting objective. More complex time-series or weather-driven models were not adopted because the available data window is limited and the forecast target is a single monthly maximum rather than a full hourly load profile. The final model therefore prioritizes robustness and interpretability while remaining consistent with the empirical backtest results.


In [6]:
# Question 1(e): Reproducible Forecast Script for March 2026 CAISO Peak Load

import pandas as pd

df = pd.read_csv("/content/CAISOHourlyLoadCSV.csv")

df["Date"] = pd.to_datetime(df["Date"])
df["Hour"] = df["Hour"].astype(int)
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month

load_col = "CAISO Load (MW)"

march_2025 = df[(df["Year"] == 2025) & (df["Month"] == 3)]
peak_idx = march_2025[load_col].idxmax()
peak_row = march_2025.loc[peak_idx]

L_2025 = float(peak_row[load_col])
peak_date_2025 = peak_row["Date"].date()

g = 0.015
L_2026 = L_2025 * (1 + g)
d_star = peak_date_2025.replace(year=2026)

print("March 2026 CAISO Peak Load Forecast")
print(f"Forecasted peak load (MW): {int(round(L_2026))}")
print(f"Forecasted peak date: {d_star}")

March 2026 CAISO Peak Load Forecast
Forecasted peak load (MW): 28549
Forecasted peak date: 2026-03-14


## Question 1 Part F:
No, I have only used the dataset provided.

## Question 1 Part G:
Not Applicable

## Question 1 Part H:
I have attached all the code and answers in this notebook file.

## Question 2

The goal of the forecasting system is to produce a reliable forecast of the CAISO system’s monthly peak load on a rolling basis, using only information that is available at the time the forecast is made. Each forecast is generated mid-month for the following calendar month and is updated as new data becomes available.

The system begins with automated data ingestion. On a fixed schedule each month, the system retrieves hourly CAISO system load data up to the most recent cutoff date. For example, the April peak forecast generated on March 15 uses hourly load data available through March 14. The data are validated for completeness, time consistency, and missing or duplicate hours before being stored in a historical database.

Once the data are ingested, the system aggregates hourly loads into monthly peak values by computing the maximum hourly load observed in each completed month. These monthly peak values form the primary input to the forecasting model, since the operational objective is to forecast the next month’s maximum load rather than average or total demand.

The forecasting step uses a parsimonious model that projects the next month’s peak load based on recent historical monthly peaks. The model estimates growth or trend parameters using only data available prior to the forecast cutoff date. Model parameters are re-estimated each month as new data become available, ensuring that the forecast reflects the most recent demand patterns without relying on future information.

The system operates on a rolling timeline. On March 15, the system produces a forecast for April peak load using data through March 14. On April 15, it produces a forecast for May peak load using data through April 14, and this process continues indefinitely. Each forecast is stored along with its issue date, forecast horizon, and underlying model parameters to allow for later evaluation.

After the actual monthly peak load is observed, the system performs backtesting and performance monitoring. Forecast errors are computed using metrics such as absolute error and percentage error. These errors are tracked over time to assess model stability and identify periods where forecast performance degrades, such as during extreme weather months. If persistent bias or large errors are observed, model assumptions or parameters can be adjusted.

Conceptually, the system workflow is as follows:
Hourly CAISO load data (through cutoff date)
→ Data validation and cleaning
→ Monthly peak extraction
→ Model estimation using historical peaks
→ Next-month peak load forecast
→ Error tracking and model updates

This system design prioritizes transparency, reproducibility, and operational feasibility while remaining flexible enough to incorporate additional data sources or more complex models in the future if warranted.

## Question 3(a)

I would forecast the 24 hourly shape factors by separating the “scale” of the peak day from the “shape” of the 24-hour profile. The scale is the peak value L* for March 2026 (from Question 1a), and the shape is a normalized 24-hour curve for the forecasted peak date d* (from Question 1b). The output I need is s_h = L_h / L* for h = 1,…,24, with max_h s_h = 1.00.


I would first  create a small library of historical March days that are similar to the forecasted peak day d*. The simplest version is to use the most recent March peak day as a template (for example, March 14, 2025 is known to be the March 2025 peak day in the provided dataset). A stronger version is to include multiple “high-load March days” such as the top K daily peaks in March across available years, and treat their hourly profiles as candidate shapes.

then convert each candidate day into shape factors
For each candidate day, I would compute that day’s hourly loads L_h,obs and normalize by that day’s maximum hourly load L_obs:

s_h,obs = L_h,obs / L_obs.

This guarantees internal consistency because the maximum hour has s_h = 1.00 and all other hours are less than or equal to 1.00. This also matches the comparison numbers given in the prompt (e.g., for 3/14/2025, Hour 11 has shape 1.00 by definition, and other hours have shape values like 0.78, 0.94, 0.93).


Next, to get a single forecasted 24-hour shape, I would combine the candidate shapes using one of the following simple rules:

Nearest-neighbor template: pick the single most similar historical day and use its shape factors directly.
Weighted average: take a weighted average of shape factors across the candidate days (weights based on similarity to d*), and then renormalize so the max is exactly 1.00.

Cluster-and-centroid: cluster historical March daily shapes and use the centroid of the cluster that contains the most extreme/high-load March days, then renormalize.

Finally, convert shape factors to hourly MW if needed
If I also want an hourly load forecast for the day, I would scale the shape factors by the forecasted peak L*:
L_h,forecast = s_h × L*.

This ensures max_h L_h,forecast = L* automatically.
Diagram (conceptual workflow)
Historical hourly CAISO load data

→ select “similar” March days (including prior March peak days / top-K high-load days)

→ compute daily peak L_obs and normalize to get s_h,obs = L_h,obs / L_obs

→ combine/average selected shapes and renormalize so max_h s_h = 1.00

→ output the 24 shape factors s_1,…,s_24 for Date = d*
(optional) scale by L* to get hourly MW: L_h = s_h × L*

## Question 3(b)

If the goal changes from forecasting CAISO’s shape factors to forecasting the LSE’s shape factors, the overall structure stays the same (normalize a 24-hour profile by its own daily maximum), but the data source and similarity logic must be based on the LSE’s load, not CAISO’s system load.

Concretely, instead of building the shape library from CAISO hourly loads, I would build it from the LSE’s hourly loads. I would compute LSE_h,obs for each candidate day, compute LSE_obs = max_h LSE_h,obs, and then define the LSE shape factors as s_h = LSE_h,obs / LSE_obs. Then I would select similar days and combine shapes using the same template/weighted-average/cluster method as in 3(a), but everything is done on the LSE series.

The main practical change is that “similarity” should reflect drivers that matter for the LSE specifically (customer mix, weekday/weekend effects, local weather zone, DER/solar behind-the-meter impacts, EV charging behavior), because those drivers can shift the intraday shape even if the CAISO system shape looks different.

If the LSE profile differs systematically from CAISO (for example, more residential load and more evening peak), the LSE’s historical shapes will encode that difference and the resulting forecasted s_h curve will shift accordingly (e.g., relatively lower midday factors and relatively higher evening factors compared to CAISO).

In [7]:
# Question 3(c): Forecasting 24 Shape Factors Using March 14, 2025 Peak-Day Profile

import pandas as pd

df = pd.read_csv("/content/CAISOHourlyLoadCSV.csv")

df["Date"] = pd.to_datetime(df["Date"])
df["Hour"] = df["Hour"].astype(int)

load_col = "CAISO Load (MW)"

# Select the known March 2025 peak day
peak_day = pd.Timestamp("2025-03-14")
day_df = df[df["Date"] == peak_day].copy()

# Compute daily peak
L_star_2025 = day_df[load_col].max()

# Compute shape factors
day_df["shape_factor"] = day_df[load_col] / L_star_2025

# Sort by hour and keep relevant columns
shape_factors = day_df.sort_values("Hour")[["Hour", "shape_factor"]]

print(f"Observed March 14, 2025 peak load (MW): {L_star_2025:.2f}")
print("Maximum shape factor:", shape_factors["shape_factor"].max())
print("\n24 hourly shape factors (s_h):")

shape_factors.reset_index(drop=True, inplace=True)
shape_factors

Observed March 14, 2025 peak load (MW): 28127.03
Maximum shape factor: 1.0

24 hourly shape factors (s_h):


,Hour,shape_factor
0,1,0.806731
1,2,0.783013
2,3,0.773111
3,4,0.760186
4,5,0.762368
5,6,0.802697
6,7,0.878739
7,8,0.943936
8,9,0.980141
9,10,0.984964


## Question 3 Part c explanation:

I implemented the shape-factor forecast described in Task 3(a) using the observed hourly load profile from March 14, 2025, which is the month-peak day identified in the provided dataset. This date represents the most recent March system peak and therefore provides a reasonable empirical template for the intraday shape of the March peak load.

I first extracted the 24 hourly CAISO system loads for March 14, 2025 and identified the daily peak load as 28,127.03 MW, which occurs at Hour 11. I then computed the 24 shape factors as s_h = L_h / L*, where L_h is the observed CAISO load at hour h on that date and L* is the daily peak load. By construction, the maximum shape factor equals 1.00 at the peak hour, and all other hours have shape factors less than or equal to 1.00, satisfying the internal consistency requirement specified in the prompt.

The resulting shape factors follow the expected intraday pattern for a March peak day, with lower values overnight and early morning, rising through the morning hours, reaching a maximum at late morning (Hour 11), and then gradually declining through the afternoon and evening. The computed values are consistent with the comparison examples provided in the assignment, such as a shape factor of approximately 0.78 at Hour 2, approximately 0.94 at Hour 8, and approximately 0.93 at Hour 13.

These 24 shape factors can be directly applied to the March 2026 peak load forecast L* by scaling each factor as L_h = s_h × L*, producing an internally consistent hourly load forecast for the peak day d*. This approach assumes that the intraday shape of the March system peak is relatively stable from year to year, while the overall magnitude of load changes according to the peak forecast.


4(a)

No.

4(b)

Not applicable.

4(c)

Yes.

4(d)

ChatGPT (version 5.2). The tool was used to draft and refine written explanations, assist with structuring responses, and debug and verify Python code used for data processing and analysis. No external solutions were copied, and all modeling decisions and final outputs reflect my own independent work.

4(e)

No.

4(f)

Not applicable.